This notebook addresses the question, "Can we represent a molecule as a graph via a 1D column vector or a 2D matrix of fixed length, with maximum number of atoms n_rows?" Then, can we use this representation to learn neural fingerprints? E.g., can we make an aromatic ring detector? 

Scheme:
feature_matrix = X
for each ligand:
    choose a central atom. this can be the atom (node) that minimizes distance to furthest heavy atom in graph.
    set first row of X to be this central atom
    set next four rows to be the atoms bonded to that centrl atom
        set zeros for rows where row ind > n_bonds of atom
    for each of those atoms:
        repeat. find their neighbors. add to matrix.

algorithm: breadth-first search:
1. create networkx graph based on molecule
2. find "central" atom (different strategies)
3. define atom matrix of size (1+4+4*3^(L-1)) x (n_features_per_atom)
4. start atom queue q
5. central_atom.layer = 0; central_atom.row_idx = 0;
6. q.enqueue(central_atom)
7. define adjacency matrix of size (1+4+4*3^(L-1)) x 4

def get_row_idx(curr_layer, prev_row_idx, curr_neighbor_idx):
    if curr_layer == 0:
        return(0)
    if curr_layer == 1:
        row_idx = 1 + curr_neighbor_idx
    if layer == 2:
        last_max = 5
        row_idx = last_max + (3*(prev_row_idx-last_max)) + curr_neighbor_idx
    if layer > 2:
        last_max = 5 + 4*3^(curr_layer-2) 
        row_idx = last_max + 3*(prev_row_idx-last_max) + curr_neighbor_idx
    return(row_idx)
    

while q.is_not_empty():
    a = q.dequeue()
    a.visited = True
    for n_idx, n in enumerate(a.neighbors()):
        if not n.visited:
            row_idx = c
            n.layer = a.layer + 1
            row_idx = get_row_idx(n.layer, a.row_idx, n_idx)
            n.row_idx = row_idx
            adj_matrix[a.row_idx][n_idx] = n.row_idx
            atom_matrix[row_idx][elem_to_idx[n.elem]] = 1

input_matrix = tf.concat([atom_matrix, atom_matrix[adj_matrix[:,0]], atom_matrix[adj_matrix[:,1]], atom_matrix[adj_matrix[:,2]], atom_matrix[adj_matrix[:,3]]

neural net:
h1 = relu([tf.zeros([n_features_per_atom, 4]) * input_matrix + bias))
h1_conc = tf.concat([h1, h1[adj_matrix[:,0], ..., h1[adj_matrix[:,3])

repeat h1 to get h2


dihedral predictor pseudocode:

get bonds for molecule
create networkx graph out of molecule (use atom indices)

for each edge:
   for neighbor_i in atom_i.neighbors():
       if neighbor_i == atom_j: continue
       for neighbor_j in atom_j.neighbors():
           if neighbor_j == atom_i: continue
           dihedrals.append((neighbor_i, atom_i, neighbor_j, atom_j))
           check to make sure (atom_j, neighbor_j, atom_i, neighbor_i)) not already in list

for dihedral in dihedrals:
    angle =  rdMolTransforms.GetDihedralDeg(c, 0,1,2,3)


In [1]:
from model import DCGAN
gan = DCGAN(n_layers=2, batch_size=8,
                                 n_atom_types=75,
                                 max_n_atoms=200,
                                 max_valence=6,
                                 n_tasks=1,
                                 learning_rate=1e-3,
                                 L_list = [5, 5, 5, 5],
                                 epsilon=1e-8,
                                 beta1=0.9,
                                 dropout=1.0)

layer_idx: 0
layer_idx: 1
layer_idx: 0
layer_idx: 1
output
Tensor("discriminator/MatMul_38:0", shape=(?, 1), dtype=float32)
non_zero_inds
Tensor("Placeholder_2:0", shape=(?, 2000), dtype=int32)
label_placeholder
Tensor("label_placeholder:0", shape=(2000,), dtype=float32)
fc_diff2
Tensor("discriminator/Sigmoid_3:0", shape=(?, 1), dtype=float32)
layer_idx: 0
layer_idx: 1
output
Tensor("discriminator_1/MatMul_38:0", shape=(?, 1), dtype=float32)
non_zero_inds
Tensor("Placeholder_2:0", shape=(?, 2000), dtype=int32)
label_placeholder
Tensor("label_placeholder:0", shape=(2000,), dtype=float32)
fc_diff2
Tensor("discriminator_1/Sigmoid_3:0", shape=(?, 1), dtype=float32)
self.G
Tensor("generator/MatMul_38:0", shape=(?, 1), dtype=float32)
self.D_logits
Tensor("discriminator/Sigmoid_3:0", shape=(?, 1), dtype=float32)
self.D_logits_
Tensor("discriminator_1/Sigmoid_3:0", shape=(?, 1), dtype=float32)


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolTransforms
import os
import fnmatch
import numpy as np
import deepchem as dc
from scipy.sparse import csr_matrix

In [3]:
def get_torsions_angles(mol):
    torsion_tuples = []
    for bond in mol.GetBonds():
        atom_i = bond.GetBeginAtom()
        atom_j = bond.GetEndAtom()
        if atom_i.IsInRing() or atom_j.IsInRing():
            continue
        for neighbor_i in atom_i.GetNeighbors():
            if neighbor_i.GetIdx() == atom_j.GetIdx():
                continue
            
            for neighbor_j in atom_j.GetNeighbors():
                if neighbor_j.GetIdx() == atom_i.GetIdx():
                    continue
                torsion_tuple = (neighbor_i.GetIdx(), atom_i.GetIdx(), atom_j.GetIdx(), neighbor_j.GetIdx())
                reverse_torsion_tuple = (neighbor_j.GetIdx(), atom_j.GetIdx(), atom_i.GetIdx(), neighbor_i.GetIdx())
                if torsion_tuple not in torsion_tuples and reverse_torsion_tuple not in torsion_tuples:
                    torsion_tuples.append(torsion_tuple)
    c = mol.GetConformer(0)
    torsions = []
    torsion_matrix = np.zeros((250,1))
    torsion_indices = np.zeros((250,200,4)).astype(np.uint8)
    for i, torsion_tuple in enumerate(torsion_tuples):
        torsion_matrix[i] = np.cos(rdMolTransforms.GetDihedralRad(c, *torsion_tuple))
        torsion_indices[i][torsion_tuple[0]][0] = 1
        torsion_indices[i][torsion_tuple[1]][1] = 1
        torsion_indices[i][torsion_tuple[2]][2] = 1
        torsion_indices[i][torsion_tuple[3]][3] = 1
    return((torsion_indices, csr_matrix(torsion_matrix)))
                

In [4]:
def featurize_mols(mol_files):
    featurizer = AdjacencyFingerprint(max_n_atoms=200)
    features = []
    for mol_file in mol_files:
        mol = Chem.MolFromMol2File(mol_file)
        if mol is None:
            features.append(None)
            continue
        torsions = get_torsions_angles(mol)
        graph_feat = featurizer.featurize([mol])[0]
        features.append((mol_file, torsions, graph_feat))
    return(features)

In [5]:
from deepchem.feat.graph_features import ConvMolFeaturizer
from deepchem.feat.adjacency_fingerprints import AdjacencyFingerprint

In [6]:
import pickle
feature_file = "../../dihedral_features_pdbbind.pkl"
if not os.path.exists(feature_file):
#if 1== 1:
    pdbbind_dir = "/home/evan/Documents/deep_docking/datasets/v2015/"
    def find_files(directory, pattern):
        for root, dirs, files in os.walk(directory):
            for basename in files:
                if fnmatch.fnmatch(basename, pattern):
                    filename = os.path.join(root, basename)
                    yield filename
    ligand_files = [f for f in find_files(pdbbind_dir, "*ligand.mol2")]
    features = featurize_mols(ligand_files)
    with open(feature_file, "wb") as f:
        pickle.dump(features, f, protocol=2)
else:
    with open(feature_file, "rb") as f:
        features = pickle.load(f)

In [7]:
features = [f for f in features if f is not None and len(np.where(f[1][1].toarray() == 0)[0]) < 249][:128]

In [8]:
len(features)

128

In [9]:
gan.train(features, n_epochs=2)

None
Training epoch 0
0.693147
1.03722
0.795927
0.693147
0.974003
0.840682
0.693147
0.971775
0.791057
0.693147
0.960818
0.805228
0.693147
0.895004
0.757661
0.693147
0.950179
0.739916
0.693147
0.921993
0.734952
0.693147
0.94995
0.794051
0.693147
0.889134
0.686285
0.693147
0.964017
0.691237
0.693147
0.917818
0.673243
0.693147
0.927528
0.712264
0.693147
0.879684
0.682508
0.693147
0.924576
0.654227
0.693147
0.884654
0.690664
0.693147
0.860562
0.697586
Training epoch 1
0.693147
0.852438
0.65835
0.693147
0.888732
0.653523
0.693147
0.882755
0.653646
0.693147
0.845675
0.663123
0.693147
0.869909
0.648544
0.693147
0.865237
0.653236
0.693147
0.867573
0.651325
0.693147
0.914809
0.640943
0.693147
0.765696
0.566176
0.693147
0.858225
0.59955
0.693147
0.833858
0.607198
0.693147
0.815384
0.61436
0.693147
0.82949
0.598372
0.693147
0.7666
0.564847
0.693147
0.817019
0.597777
0.693147
0.818806
0.594533


In [10]:
gan.predict(features)

layer_idx: 0
layer_idx: 1


FailedPreconditionError: Attempting to use uninitialized value generator_1/Variable_2
	 [[Node: generator_1/Variable_2/read = Identity[T=DT_FLOAT, _class=["loc:@generator_1/Variable_2"], _device="/job:localhost/replica:0/task:0/gpu:0"](generator_1/Variable_2)]]
	 [[Node: generator_1/MatMul_38/_95 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1_generator_1/MatMul_38", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'generator_1/Variable_2/read', defined at:
  File "/home/evan/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/evan/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/evan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instan

In [ ]:
len(features)

In [10]:
features[9][1][1]

array([[-0.04800897],
       [-0.07369911],
       [-0.85061992],
       [-0.25623746],
       [-0.99060946],
       [ 0.32041239],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0

In [ ]:
features[0][2].atom_features